In [1]:
import pandas as pd
from pathlib import Path

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# Directory of this file
this_dir = Path('.').resolve()
this_dir

PosixPath('/home/jovyan/work/my_projects/in_progress/Python_in_Excel/Pandas_in_Excel')

In [3]:
# Read in all Excel files from all subfolders of sales data
parts = []
for path in (this_dir / 'sales_data/new').rglob('*.xls*'):
    print(f'Reading {path.name}')
    part = pd.read_excel(path, index_col='transaction_id')
    parts.append(part)

Reading February.xlsx
Reading August.xlsx
Reading April.xlsx
Reading June.xlsx
Reading October.xlsx
Reading September.xlsx
Reading July.xlsx
Reading January.xlsx
Reading May.xlsx
Reading November.xlsx
Reading December.xlsx
Reading March.xlsx


In [4]:
# Combine the DataFrames from each file into a single DataFrame
# Pandas takes care of properly aligning the columns
df = pd.concat(parts)
df

,store,status,transaction_date,plan,contract_type,amount
transaction_id,,,,,,
629b9f16,New York,ACTIVE,2019-02-01,Silver,NEW,14.25
2f57ac6c,Las Vegas,ACTIVE,2019-02-01,Silver,NEW,14.25
45e063e6,Las Vegas,ACTIVE,2019-02-01,Silver,NEW,14.25
6ef3a43a,New York,ACTIVE,2019-02-01,Silver,NEW,14.25
89f0cec1,San Francisco,ACTIVE,2019-02-01,Gold,NEW,19.35
...,...,...,...,...,...,...
02050375,Las Vegas,ACTIVE,2019-03-29,Silver,NEW,14.25
18353e1e,San Francisco,ACTIVE,2019-03-29,Bronze,NEW,12.20
8d1f10ff,San Francisco,ACTIVE,2019-03-29,Silver,NEW,14.25


In [10]:
# Pivot each store into a column and sum up all transactions per date
pivot = pd.pivot_table(df, index='transaction_date', columns='store', values='amount', aggfunc='sum')
pivot

store,Boston,Chicago,Las Vegas,New York,San Francisco,Washington DC
transaction_date,,,,,,
2019-01-01,465.15,1277.25,269.70,1740.25,1739.20,355.15
2019-01-02,621.80,1177.50,369.45,1833.85,1861.45,326.70
2019-01-03,449.85,1204.95,302.25,1614.05,1736.20,286.95
2019-01-04,600.45,1165.25,340.95,1712.95,1763.75,204.55
2019-01-07,599.40,1197.80,395.90,1427.70,1967.20,265.65
...,...,...,...,...,...,...
2019-12-25,499.75,894.55,282.95,1480.95,1575.40,224.85
2019-12-26,382.65,971.85,298.25,1260.90,1427.75,193.35
2019-12-27,525.20,821.30,262.55,1429.75,1758.45,209.60


In [11]:
# Resample to the end of month and assign an index name
summary = pivot.resample('ME').sum()
summary.index.name = 'Month'
summary

store,Boston,Chicago,Las Vegas,New York,San Francisco,Washington DC
Month,,,,,,
2019-01-31,13701.10,26272.65,7216.45,42128.25,42969.25,6515.40
2019-02-28,12243.85,24739.90,6192.60,37926.55,36469.35,6705.70
2019-03-31,11026.60,22131.00,5637.25,32805.60,33876.45,5173.80
2019-04-30,10344.80,21489.35,5540.40,32954.35,33265.00,5480.30
2019-05-31,10533.15,20606.55,5266.45,32348.55,31836.80,5293.95
2019-06-30,13317.90,24328.15,5713.55,38543.65,37787.30,5926.05
2019-07-31,10701.65,20877.90,5376.65,31547.60,31386.65,4982.55
2019-08-31,12239.70,24662.75,6335.05,36491.60,36895.10,6729.15
2019-09-30,10757.10,20964.35,5928.55,33226.20,31709.80,5544.45


In [12]:
# Add row and column totals
summary.loc[:, "Total"] = summary.sum(axis=1)
summary.loc['Total', :] = summary.sum(axis=0)
summary

store,Boston,Chicago,Las Vegas,New York,San Francisco,Washington DC,Total
Month,,,,,,,
2019-01-31 00:00:00,13701.10,26272.65,7216.45,42128.25,42969.25,6515.40,138803.10
2019-02-28 00:00:00,12243.85,24739.90,6192.60,37926.55,36469.35,6705.70,124277.95
2019-03-31 00:00:00,11026.60,22131.00,5637.25,32805.60,33876.45,5173.80,110650.70
2019-04-30 00:00:00,10344.80,21489.35,5540.40,32954.35,33265.00,5480.30,109074.20
2019-05-31 00:00:00,10533.15,20606.55,5266.45,32348.55,31836.80,5293.95,105885.45
2019-06-30 00:00:00,13317.90,24328.15,5713.55,38543.65,37787.30,5926.05,125616.60
2019-07-31 00:00:00,10701.65,20877.90,5376.65,31547.60,31386.65,4982.55,104873.00
2019-08-31 00:00:00,12239.70,24662.75,6335.05,36491.60,36895.10,6729.15,123353.35
2019-09-30 00:00:00,10757.10,20964.35,5928.55,33226.20,31709.80,5544.45,108130.45


## Write summary report to Excel file

In [13]:
# DataFrame position and number of rows/columns
# xlsxwriter uses 0-based indices
startrow, startcol = 2, 1
nrows, ncols = summary.shape

In [14]:
with pd.ExcelWriter(this_dir / "sales_report_xlsxwriter.xlsx",
                    engine="xlsxwriter", datetime_format="mmm yy") as writer:
    summary.to_excel(writer, sheet_name="Sheet1",
                     startrow=startrow, startcol=startcol)

    # Get xlsxwriter book and sheet object
    book = writer.book
    sheet = writer.sheets["Sheet1"]

    # Set title
    title_format = book.add_format({"bold": True, "size": 24})
    sheet.write(0, startcol, "Sales Report", title_format)

    # Sheet formatting
    # 2 = hide on screen and when printing
    sheet.hide_gridlines(2)

    # Format the DataFrame with
    # - number format
    # - column width
    # - conditional formatting
    number_format = book.add_format({"num_format": "#,##0",
                                     "align": "center"})
    below_target_format = book.add_format({"font_color": "#E93423"})
    sheet.set_column(first_col=startcol, last_col=startcol + ncols,
                     width=14, cell_format=number_format)
    sheet.conditional_format(first_row=startrow + 1,
                             first_col=startcol + 1,
                             last_row=startrow + nrows,
                             last_col=startcol + ncols,
                             options={"type": "cell", "criteria": "<=",
                                      "value": 20000,
                                      "format": below_target_format})

    # Chart
    chart = book.add_chart({"type": "column"})
    chart.set_title({"name": "Sales per Month and Store"})
    chart.set_size({"width": 830, "height": 450})

    # Add each column as a series, ignoring total row and col
    for col in range(1, ncols):
        chart.add_series({
            # [sheetname, first_row, first_col, last_row, last_col]
            "name": ["Sheet1", startrow, startcol + col],
            "categories": ["Sheet1", startrow + 1, startcol,
                           startrow + nrows - 1, startcol],
            "values": ["Sheet1", startrow + 1, startcol + col,
                       startrow + nrows - 1, startcol + col],
        })

    # Chart formatting
    chart.set_x_axis({"name": summary.index.name,
                      "major_tick_mark": "none"})
    chart.set_y_axis({"name": "Sales",
                      "line": {"none": True},
                      "major_gridlines": {"visible": True},
                      "major_tick_mark": "none"})

    # Add the chart to the sheet
    sheet.insert_chart(startrow + nrows + 2, startcol, chart)